## Introduction

"The precision matrix of a random vector is the inverse of its covariance matrix." The covariance matrix is dense, perhaps without a single zero. The precision matrix is sparse and may be generated without much trouble.

Wind farms are correlated both spatially and temporally. We must arrange the farms in a tiled pattern so each site's neighbors may be classified as North, South, West, or East. Then we can assign coefficients to weight spatial and temporal correlation. Using these coefficients and assuming our model is first-order in time (tri-diagonal precision matrix blocks), we can construct a precision matrix.

## Precision matrix for RTS-96

The wind sites in the ARPA-E RTS-96 are distributed within three areas. We will assume those areas are distant enough to be uncorrelated. Within each area, the spatial arrangement is arbitrary.

There are nine wind sites in Area 1, six in Area 2, and three in Area 3. I tiled the sites in each area as shown below.

![](../images/rts96adjacency.svg)

To encode these spatial relationships, I will use the custom type `WindAdj`. This type has five fields, each containing a bus index (integer).

![](../images/windadj.svg)

In [25]:
type WindAdj
    A::Int64
    N::Int64
    E::Int64
    S::Int64
    W::Int64
end

In [27]:
windsites = Vector{WindAdj}()
push!(windsites,WindAdj( 2, 0,14, 0, 0))
push!(windsites,WindAdj(14, 0,16,17, 2))
push!(windsites,WindAdj(16, 0, 0,18,14))
push!(windsites,WindAdj(17,14,18,20, 0))
push!(windsites,WindAdj(18,16,19, 0,17))
push!(windsites,WindAdj(19, 0, 0,21,18))
push!(windsites,WindAdj(20,17, 0, 0, 0))
push!(windsites,WindAdj(21,19,23, 0, 0))
push!(windsites,WindAdj(23, 0, 0, 0,21))
push!(windsites,WindAdj(26, 0, 0,36, 0))
push!(windsites,WindAdj(36,26,37,43, 0))
push!(windsites,WindAdj(37, 0, 0,44,36))
push!(windsites,WindAdj(43,36,44,47, 0))
push!(windsites,WindAdj(44,37, 0, 0,43))
push!(windsites,WindAdj(47,43, 0, 0, 0))
push!(windsites,WindAdj(49, 0,54, 0, 0))
push!(windsites,WindAdj(54, 0, 0,57,49))
push!(windsites,WindAdj(57,54, 0, 0, 0));

## Precision matrix generation

Take a set of coefficients and the spatial information just described, and generate a precision matrix.

According to Tastu2015, the precision matrix $Q$ "can be fully modeled by a parameter vector $\theta$, as

$$ \theta = \left[ \kappa_1,\rho,\kappa_K,\sigma^2,a_{-1},b_0,b_{-1},b_1,c_0,c_{-1},c_1\right]^\top. $$

"It will hence be referred to as $Q(\theta)$..."

The paper breaks down $Q(\theta)$ as $Q=\kappa B$. The block-diagonal matrix $\kappa$ contains conditional precisions. The symmetric positive-definite coefficient matrix $B$ may be viewed as a standardized precision matrix.

### Coefficient matrix $\kappa$

The matrix $\kappa$ is block-diagonal, appearing as

$$ \kappa = \begin{bmatrix} \kappa_B & & & 0 \\
& \kappa_B & & \\
& & \ddots & \\
0 & & & \kappa_B \end{bmatrix}, $$

where each block is a repeat of the same $\kappa_B$:

$$ \kappa_B = \begin{bmatrix} \kappa_1 & & & 0 \\
& \kappa_2 & & \\
& & \ddots & \\
0 & & & \kappa_K \end{bmatrix}. $$

Temporal boundaries correspond to $\kappa_1$ and $\kappa_K$. These precision values will be smaller (see Eqs. 6 and 7 on pg. 9 of Tastu2015). Remaining values $\kappa_2$ to $\kappa_{K-1}$ increase with lead time. A suitable model for the increase is

$$ \kappa_i = \rho^{i-2},~~ i=2,\ldots,K-1 $$

Putting the pieces together, we obtain the following parametrization of $\kappa_B$ in terms of four parameters ($\kappa_1$, $\kappa_k$, $\rho$, and $\sigma^2$):

$$ \kappa_B = \frac{1}{\sigma^2} \begin{bmatrix}
\kappa_1 & & & & & \\
& 1 & & & 0 & \\
& & \rho & & & \\
& & & \ddots & & \\
& 0 & & & \rho^{K-2} & \\
& & & & & \kappa_K \end{bmatrix}. $$

In [11]:
""" For calculating a precision matrix.
Return a sparse diagonal coefficient matrix kappaB based on
the four parameters described in Tastu2015 pg. 13.
"""
function kappaB(K,kappa1,kappaK,rho,sigma)
    d = [kappa1;[rho^(i-2) for i in 2:(K-1)];kappaK]
    return spdiagm(d)/(sigma^2)
end

kappaB (generic function with 1 method)

In [13]:
full(kappaB(5,9,9,0.5,3))

5x5 Array{Float64,2}:
 1.0  0.0       0.0        0.0        0.0
 0.0  0.111111  0.0        0.0        0.0
 0.0  0.0       0.0555556  0.0        0.0
 0.0  0.0       0.0        0.0277778  0.0
 0.0  0.0       0.0        0.0        1.0

### Standardized precision matrix $B$

